# 2D flying-snake at Re=1000 and AoA=35deg (snappyHexMesh + OpenFOAM-2.3.0)

---

This notebook heavily relies on the Python package `snake`, available on [GitHub](https://github.com/mesnardo/snake).
`snake` was written to post-process and compare the numerical solution from 4 different softwares ([`cuIBM`](https://github.com/barbagroup/cuIBM), [`PetIBM`](https://github.com/barbagroup/PetIBM), [`IBAMR`](https://github.com/IBAMR/IBAMR), and [`OpenFOAM`](http://www.openfoam.com/)).

In [1]:
import os
import sys
import collections

import ipywidgets

import snake
from snake import miscellaneous
from snake.geometry import Geometry
from snake.openfoam import OBJFile
from snake.openfoam.simulation import OpenFOAMSimulation
from snake.cuibm.simulation import CuIBMSimulation

In [2]:
print('Python version: ' + sys.version)
print('snake version:' + snake.__version__)

Python version: 2.7.11 |Anaconda 2.5.0 (64-bit)| (default, Dec  6 2015, 18:08:32) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
snake version:0.1


---

### Description

We aim to compute the 2D flow around a snake cross-section with angle-of-attack 35 degrees at Reynolds number 1000 using IcoFOAM, the laminar incompressible solver of OpenFOAM.

The computational mesh, generated with the OpenFOAM mesh utility snappyHexMesh, contains about 3.4 million cells.

---

### General information

* software: `OpenFOAM-2.3.0`
* post-processing software: `ParaView-4.1` (included in `OpenFOAM-2.3.1`)
* submission date: 2015/10/30 (mesnardo)
* running date: 2015/10/30
* machine: [`Colonial One`](https://ots.columbian.gwu.edu/colonial-one-high-performance-computing-initiative) (short queue, 16 processes, 1 node)

In [ ]:
simulation = OpenFOAMSimulation(description='IcoFOAM')

---

### Computational domain and mesh

The mesh contains about 3.4 million cells and was created with the 3D OpenFOAM mesh utility, `snappyHexMesh`.

Details about the mesh-generation process are available [here](../meshDetails.ipynb).

To generate the mesh, you need first to create the necessary .obj files (`snake.obj`, `boxNear.obj`, and `boxWake.obj`). To do that, run the cell below.
Afterwards, you can generate the mesh with the command-line:
    > runSnappyHexMesh.sh

In [ ]:
# create triSurface directory if not present
save_directory = os.path.join(simulation.directory,
                              'constant',
                              'triSurface')
if not os.path.isdir(save_directory):
    os.makedirs(save_directory)

# save the bluff-body as .obj file
body = Geometry(file_path=os.path.join(os.environ['SNAKE'],
                                       'resources',
                                       'geometries',
                                       'flyingSnake2d',
                                       'flyingSnake2dAoA35.dat'))
body.write('tmpBody.dat')
body = OBJFile.Body2d('tmpBody.dat',
                      name='snake',
                      extrusion_limits=(0.0, 1.0))
body.write(save_directory=save_directory)
os.remove('tmpBody.dat')

# create two boxes of refinement as .obj files
box = OBJFile.Box2d('boxNear',
                    bottom_left=(-1.0, -2.0),
                    top_right=(10.0, 2.0),
                    n=(800, 400))
box.write(save_directory)
box = OBJFile.Box2d('boxWake',
                    bottom_left=(-2.0, -4.0),
                    top_right=(15.0, 4.0),
                    n=(700, 300))
box.write(save_directory)

---

### Boundary conditions

In [ ]:
boundary_info = '0/U'
%cat $boundary_info
boundary_info = '0/p'
%cat $boundary_info

---

### Simulation parameters

In [ ]:
simulation_info = 'system/controlDict'
%cat $simulation_info
simulation_info = 'system/fvSolution'
%cat $simulation_info
simulation_info = 'system/fvScheme'
%cat $simulation_info

---

### Running the case

To run the the case in parallel , we need to decompose the domain:
    > decomposeParDict

To run the case the simulation:
    > mpirun -n <nprocs> icoFOAM -parallel > log.icoFoam

To reconstruct the parallel solution:
    > reconstructPar

To generate the vorticity field at each time saved:
    > vorticity

---

### Post-processing

We computed about 96 non-dimensional time-units in 48 hours.

The simulation did not complete (we requested 100 time-units) because it reached a time-limit of 48h, but we are mostly interested in what is happening up to 80 time-units.

#### Forces

We plot the instantaneous force coefficients up to 80 time-units and compute the time-averaged values between 32 and 64 time-units.

In [ ]:
simulation.read_forces(display_coefficients=True)
simulation.get_mean_forces(limits=[32.0, 64.0])
simulation.get_strouhal(limits=[32.0, 64.0], order=200)
simulation.plot_forces(display_coefficients=True,
                       display_extrema=True, order=200,
                       limits=(0.0, 80.0, 0.0, 3.0),
                       style='snakeReproducibility',
                       save_name='forceCoefficients')
dataframe = simulation.create_dataframe_forces(display_strouhal=True,
                                               display_coefficients=True)

We compare the IcoFOAM results with those reported in Krishnan et al. (2014).

In [ ]:
krishnan = CuIBMSimulation(description='Krishnan et al. (2014)')
krishnan.read_forces(file_path=os.path.join(os.environ['SNAKE'],
                                            'resources',
                                            'flyingSnake2d_cuibm_anush',
                                            'flyingSnake2dRe1000AoA35',
                                            'forces'))
krishnan.get_mean_forces(limits=[32.0, 64.0])
krishnan.get_strouhal(limits=[32.0, 64.0], order=200)
simulation.plot_forces(display_coefficients=True,
                       display_extrema=True, order=200,
                       limits=(0.0, 80.0, 0.0, 3.0),
                       other_simulations=krishnan,
                       other_coefficients=2.0,
                       style='snakeReproducibility',
                       save_name='forceCoefficientsCompareKrishnanEtAl2014')
dataframe2 = krishnan.create_dataframe_forces(display_strouhal=True,
                                              display_coefficients=True,
                                              coefficient=2.0)

In [ ]:
dataframes = dataframe.append(dataframe2)
print(dataframes)

In [ ]:
images_directory = os.path.join(simulation.directory,
                                'images')
figures = collections.OrderedDict()
figures['present'] = os.path.join(images_directory,
                                  'forceCoefficients.png')
figures['present + Krishnan et al. (2014)'] = os.path.join(images_directory,
                                                           'forceCoefficientsCompareKrishnanEtAl2014.png')
ipywidgets.interact(miscellaneous.display_image, figure=figures);

#### Vorticity field

The cells below plot the vorticity field in the region [-2,15]x[-5,5] at every saved time-unit with ParaFOAM in batch mode and display the .png files in an interactive way.

In [ ]:
simulation.plot_field_contours_paraview('vorticity',
                                        field_range=(-5.0, 5.0),
                                        view=(-2.0, -5.0, 15.0, 5.0),
                                        width=600,
                                        colormap='RdBu_r')

In [ ]:
miscellaneous.displayer(os.path.join(simulation.directory,
                                     'images',
                                     'vorticity_-2.00_-5.00_15.00_5.00'))